# RDD reconstruction example

In [ ]:
from sfm import match_rdd, extract_rdd
from hloc import (
    extract_features,
    reconstruction,
    visualization,
    pairs_from_retrieval,
    pairs_from_exhaustive,
)
from pathlib import Path
import os
import torch

In [ ]:
images_dir = Path('./assets/mapping')
device = torch.cuda.is_available()
images = [image for image in os.listdir(images_dir) if image.endswith('.jpg') or image.endswith('.png')]
outputs = Path('./outputs/reconstruction')
if not outputs.exists():
    outputs.mkdir(parents=True)
sfm_pairs = outputs / 'sfm_pairs.txt'
retrieval_conf = extract_features.confs["netvlad"]
feature_conf = extract_rdd.confs["rdd"]
matcher_conf = match_rdd.confs["rdd+lightglue"]
exhaustive_if_less = 30
num_matched = 20

In [ ]:
# image_retrieval
if len(images) < exhaustive_if_less:
    pairs_from_exhaustive.main(sfm_pairs, images)
else:
    retrieval_path = extract_features.main(retrieval_conf, images_dir, outputs)
    pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=num_matched)
    
# feature_extraction
feature_path = extract_rdd.main(feature_conf, images_dir, outputs)
# matching
match_path = match_rdd.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs)

In [ ]:
# reconstruction
image_options = {}
mapper_options = {}
model = reconstruction.main(outputs, images_dir, sfm_pairs, feature_path, 
            match_path, verbose=True, camera_mode='PER_IMAGE', image_options=image_options, mapper_options=mapper_options,
            min_match_score = 0.2, skip_geometric_verification=False)

In [ ]:
print(model.summary())